# User Behavior Overview

## Imports

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
# pd.set_option('display.float_format', '{:.2f}'.format)

In [3]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from vis_plotly import *

## Data reading

In [4]:
df = pd.read_csv("../data/clean_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146891 entries, 0 to 146890
Data columns (total 34 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   bearer_id                146891 non-null  float64
 1   msisdn_number            146891 non-null  float64
 2   imei                     146891 non-null  float64
 3   imsi                     146891 non-null  float64
 4   last_location_name       146891 non-null  object 
 5   handset_manufacturer     146891 non-null  object 
 6   handset_type             146891 non-null  object 
 7   start                    146891 non-null  object 
 8   end                      146891 non-null  object 
 9   dur_(ms)                 146891 non-null  float64
 10  social_media_dl_(bytes)  146891 non-null  float64
 11  social_media_ul_(bytes)  146891 non-null  float64
 12  google_dl_(bytes)        146891 non-null  float64
 13  google_ul_(bytes)        146891 non-null  float64
 14  emai

##  Top 10 handsets used by the customers

In [5]:
class DfSelector():

    def __init__(self):
        pass

    def filter_by_count(self, df, column):
      new_df = df[column].value_counts(ascending=False).reset_index().copy()
      new_df = new_df.rename(
          columns={'index': column, column: "count"})
      return new_df

    def filter_by_sum(self, df, column):
      new_df = df[column].sum(ascending=False).reset_index().copy()

      new_df = new_df.rename(
          columns={'index': column, column: "sum"})
      return new_df

    
    def find_agg(self, df: pd.DataFrame, agg_column: str, agg_metric: str, col_name: str, top: int, order=False) -> pd.DataFrame:
        new_df = df.groupby(agg_column)[agg_column].agg(agg_metric).reset_index(name=col_name).\
        sort_values(by=col_name, ascending=order)[:top]
        return new_df
    
selector = DfSelector()


In [6]:
count_df = selector.filter_by_count(df, 'handset_type')
count_df.head(10)

,handset_type,count
0,Huawei B528S-23A,28193
1,Apple iPhone 6S (A1688),9369
2,Apple iPhone 6 (A1586),8967
3,Apple iPhone 7 (A1778),6240
4,Apple iPhone Se (A1723),5151
5,Apple iPhone 8 (A1905),4961
6,Apple iPhone Xr (A2105),4542
7,Samsung Galaxy S8 (Sm-G950F),4404
8,Apple iPhone X (A1901),3788
9,Samsung Galaxy A5 Sm-A520F,3673


In [7]:
fig = px.histogram(count_df.nlargest(10, 'count'), x="handset_type", y='count')
fig.show()

## Top 3 handset manufacturers

In [8]:
top_manufacturers = selector.filter_by_count(df, 'handset_manufacturer')
top_manufacturers.head(3)

,handset_manufacturer,count
0,Apple,67892
1,Samsung,39902
2,Huawei,33755


##  Top 5 handsets per top 3 handset manufacturer

In [9]:
top3_manufacturers = top_manufacturers.head(3)['handset_manufacturer'].values
top_manufacturers = df[df["handset_manufacturer"].isin(top3_manufacturers)]
top_manufacturers['handset_type'].groupby(
    df['handset_manufacturer']).apply(lambda x: x.value_counts().head(5))

handset_manufacturer                                
Apple                 Apple iPhone 6S (A1688)            9369
                      Apple iPhone 6 (A1586)             8967
                      Huawei B528S-23A                   8820
                      Apple iPhone 7 (A1778)             6240
                      Apple iPhone Se (A1723)            5151
Huawei                Huawei B528S-23A                  19373
                      Huawei E5180                       2062
                      Huawei P20 Lite Huawei Nova 3E     2000
                      Huawei P20                         1471
                      Huawei Y6 2018                      984
Samsung               Samsung Galaxy S8 (Sm-G950F)       4404
                      Samsung Galaxy A5 Sm-A520F         3673
                      Samsung Galaxy J5 (Sm-J530)        3664
                      Samsung Galaxy J3 (Sm-J330)        3431
                      Samsung Galaxy S7 (Sm-G930X)       3106
Name: handset_typ

# Task 1.1

## Number of xDR sessions

In [10]:
sessions = selector.find_agg(df, 'msisdn_number', 'count', 'bearer_id', 10, False)
sessions

,msisdn_number,bearer_id
13381,3.362632e+10,18
6353,3.361489e+10,17
13037,3.362578e+10,17
36669,3.365973e+10,16
75564,3.367588e+10,15
91949,3.376054e+10,15
64472,3.366716e+10,13
653,3.360313e+10,12
91608,3.376041e+10,12
13845,3.362708e+10,12


## Session duration

In [22]:
duration = selector.find_agg(df, 'msisdn_number', 'sum', 'dur_(ms)', -1, False)
duration.nlargest(20, 'dur_(ms)')


,msisdn_number,dur_(ms)
105718,3.370000e+14,3.370000e+14
13381,3.362632e+10,6.052738e+11
13037,3.362578e+10,5.716382e+11
6353,3.361489e+10,5.714532e+11
36669,3.365973e+10,5.385556e+11
91949,3.376054e+10,5.064080e+11
75564,3.367588e+10,5.051382e+11
64472,3.366716e+10,4.376731e+11
91608,3.376041e+10,4.051250e+11
13845,3.362708e+10,4.035250e+11


In [25]:
duration.describe()


count    1.057180e+05
mean     4.997656e+10
std      1.036670e+12
min      3.360100e+10
25%      3.365960e+10
50%      3.367059e+10
75%      6.724550e+10
max      3.370000e+14
Name: dur_(ms), dtype: float64

## The total download (DL) and upload (UL) data

In [13]:
usage = df.groupby('msisdn_number')[['total_dl_(bytes)', 'total_ul_(bytes)']].sum()
usage['total_data'] = usage['total_dl_(bytes)'] + usage['total_ul_(bytes)']
usage.nlargest(20, 'total_data')

,total_dl_(bytes),total_ul_(bytes),total_data
msisdn_number,,,
3.361489e+10,8.156743e+09,689483001.0,8.846226e+09
3.376054e+10,7.811295e+09,703478581.0,8.514774e+09
3.362578e+10,7.770043e+09,729577380.0,8.499621e+09
3.362632e+10,7.301517e+09,669650721.0,7.971167e+09
3.367588e+10,7.309542e+09,581568792.0,7.891111e+09
3.365973e+10,7.081602e+09,624260321.0,7.705863e+09
3.366646e+10,6.903440e+09,405060976.0,7.308501e+09
3.376041e+10,6.610852e+09,521518890.0,7.132371e+09
3.366471e+10,6.400774e+09,471244453.0,6.872018e+09


In [14]:
usage.describe()

,total_dl_(bytes),total_ul_(bytes),total_data
count,1.057190e+05,1.057190e+05,1.057190e+05
mean,6.317300e+08,5.713601e+07,6.888661e+08
std,4.609209e+08,3.527180e+07,4.869964e+08
min,8.827082e+06,2.866892e+06,3.324901e+07
25%,3.142406e+08,3.637849e+07,3.579501e+08
50%,5.693797e+08,4.676298e+07,6.169335e+08
75%,8.062981e+08,6.550410e+07,8.564738e+08
max,8.156743e+09,7.295774e+08,8.846226e+09


## The total data volume (in Bytes) during this session for each application

In [15]:
df["social_media"] = df["social_media_dl_(bytes)"] + df['social_media_ul_(bytes)']
df["google"] = df["google_dl_(bytes)"] + df["google_ul_(bytes)"]
df['email'] = df["email_dl_(bytes)"] + df["email_ul_(bytes)"]
df['youtube'] = df["youtube_dl_(bytes)"] + df["youtube_ul_(bytes)"]
df['netflix'] = df["netflix_dl_(bytes)"] + df["netflix_ul_(bytes)"]
df["gaming"] = df["gaming_dl_(bytes)"] + df["gaming_ul_(bytes)"]
df['other'] = df["other_dl_(bytes)"]+df["other_ul_(bytes)"]
df['total_data'] = df['total_dl_(bytes)'] + df['total_ul_(bytes)']

In [16]:
apps_df = df.groupby('msisdn_number')[['social_media', 'google', 'email', 'youtube', 'gaming', 'other', 'total_data']].sum().nlargest(20, 'total_data')
apps_df

,social_media,google,email,youtube,gaming,other,total_data
msisdn_number,,,,,,,
3.361489e+10,28294544.0,127973787.0,40788634.0,394370218.0,7.749432e+09,7.639264e+09,8.846226e+09
3.376054e+10,39783189.0,123223099.0,33693767.0,396289198.0,7.461045e+09,4.716134e+09,8.514774e+09
3.362578e+10,27135500.0,142307915.0,40633966.0,452958769.0,7.326673e+09,6.354583e+09,8.499621e+09
3.362632e+10,43374779.0,152191852.0,42418782.0,374483047.0,6.887572e+09,8.167878e+09,7.971167e+09
3.367588e+10,19222921.0,109860502.0,31514421.0,317410572.0,6.970568e+09,6.798515e+09,7.891111e+09
3.365973e+10,35412358.0,116516345.0,35999792.0,257991088.0,6.725559e+09,6.317415e+09,7.705863e+09
3.366646e+10,18629986.0,89320737.0,25557139.0,227336012.0,6.646303e+09,4.349141e+09,7.308501e+09
3.376041e+10,20777205.0,82738720.0,25576965.0,303169107.0,6.268620e+09,4.101645e+09,7.132371e+09
3.366471e+10,19390599.0,90389372.0,21426007.0,276834013.0,6.103856e+09,3.976960e+09,6.872018e+09


In [17]:
df.to_csv('../data/clean_data.csv', index=False)